In [42]:
%matplotlib inline
import numpy as np
from SktWsegRWR_utf8 import *

Current folder contains:  44790  Files
Loading Prob. Models
Prob. Models loaded


In [122]:
def RWR(prioriVec, transMat, restartP, maxIteration, queryList, deactivated, allowRPModify = True):
    """
    Run Random walk with restart
    until 
    we reach steady state or max iteration steps
    """
#     print(prioriVec)
    transMat = np.copy(transMat)
    
#     MERGE THE NEW QUERY NODE(IF ANY), CHANGES IN TRANSMAT AND PRIORI-VEC
    doMax = True
    if(len(queryList) > 1):
        dest = queryList[0]
        if(doMax):
            # Using the max probability logic
            transMat[dest, :] = np.max(transMat[queryList, :], axis=0)
            transMat[queryList[1:], :] = 0   
            
            transMat[:, dest] = np.max(transMat[:, queryList], axis=1)
            transMat[:, queryList[1:]] = 0
        else:
            # TODO - Using the sum probability logic
            transMat[dest, :] = np.sum(transMat[queryList, :], axis=0)
            transMat[queryList[1:], :] = 0
            
            transMat[:, dest] = np.sum(transMat[:, queryList], axis=1)
            transMat[:, queryList[1:]] = 0

    transMat[:, deactivated] = 0
    transMat[deactivated, :] = 0
    # MakeRowStochastic(transMat)


    
    eps = 0.0000001    # the error difference, which should ideally be zero but can never be attained.
    
    n = prioriVec.shape[1]
    papMat = np.copy(prioriVec)
    
    rVec = np.zeros((1, n))    
#     print(n)
    for i in queryList:
#         print(i)
        rVec[0, i] = 1/len(queryList)

    nodes = []
    for i in range(prioriVec.shape[1]):
        if(prioriVec[0,i] > 0):
            nodes.append(i)

    if(allowRPModify):
        """
        Find Dia of graph using Floyd Warshall
        Dynamically decide restart probability
        """
        dia = GetDiaFromTransmat(nodes, transMat)
        rp_new = 1 - math.pow(.045, 1/dia)
        print( "Dia: ", dia, " RP: ", rp_new)
    else:
        rp_new = restartP
    
    
    print(papMat)
    for i in range(maxIteration):        
        newMat = (1 - rp_new) * np.dot(papMat, transMat) + rp_new * np.mat(rVec)        
        diff = np.absolute(papMat - newMat)
        diffMax = np.max(diff)
        papMat = np.copy(newMat)
        print(papMat)
#         print(diffMax)
        if  abs(diffMax) < eps and i > 25:
            break
                  
    return(papMat)

In [124]:
# from numpy.random import randint
# transMat = np.zeros((5, 5))
# for i in range(5):
#     for j in range(i, 5):
#         transMat[j, i] = transMat[i, j] = randint(10)
        
#     transMat[i, i] = 0
#     transMat[i, :] /= sum(transMat[i, :])
print(transMat)

# # transMat = np.mat([
# #         [0, 0.7, 0, 0.3, 0],
# #         [0.5, 0, 0.5, 0, 0],
# #         [0, 0, 0, 0, 1],
# #         [0.2, 0, 0.8, 0, 0],
# #         [0.89, 0, 0.11, 0, 0]
# #     ])
# # transMat
# transMat = np.mat([
#         [0, 0.01, 0.98, 0, 0.01],
#         [0.45, 0, 0, 0, 0.55],
#         [0, 0, 0, 0, 0],
#         [0.6, 0.2, 0, 0, 0.2],
#         [0.8, 0, 0.2, 0, 0]
#     ])

for row in range(5):
    print(transMat[row, :])
    print(np.sum(transMat[row, :]))

[[ 0.    0.47  0.    0.37  0.16]
 [ 0.56  0.    0.12  0.25  0.06]
 [ 0.    0.17  0.    0.5   0.33]
 [ 0.32  0.18  0.27  0.    0.23]
 [ 0.23  0.08  0.31  0.38  0.  ]]
[ 0.    0.47  0.    0.37  0.16]
1.0
[ 0.56  0.    0.12  0.25  0.06]
1.0
[ 0.    0.17  0.    0.5   0.33]
1.0
[ 0.32  0.18  0.27  0.    0.23]
1.0
[ 0.23  0.08  0.31  0.38  0.  ]
1.0


In [127]:
prioriVec = np.ones((1, 5))*1/5
# prioriVec= np.ones((1, 5))
# for i in range(5):
#     prioriVec[0, i] = pv[i]

queryList = [2]

restartP = 0

maxIteration = 100

rank = RWR(prioriVec, transMat, restartP, maxIteration, queryList, deactivated=[], allowRPModify=True)
print('Final Rank')
print(rank)

Dia:  8.11111111111  RP:  0.3177277506905667
[[ 0.2  0.2  0.2  0.2  0.2]]
[[ 0.15  0.12  0.41  0.21  0.11]]
[[ 0.11  0.13  0.39  0.23  0.15]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.11  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
[[ 0.12  0.12  0.4   0.22  0.14]]
Final Ra

In [128]:
prioriVec = np.ones((1, 5))*1/5

queryList = [1, 4]

restartP = 0.1

maxIteration = 100

rank = RWR(prioriVec, transMat, restartP, maxIteration, queryList, deactivated=[], allowRPModify=False)
print('Final Rank')
print(rank)

[[ 0.2  0.2  0.2  0.2  0.2]]
[[ 0.16  0.25  0.1   0.23  0.05]]
[[ 0.19  0.21  0.12  0.19  0.05]]
[[ 0.16  0.22  0.1   0.19  0.05]]
[[ 0.17  0.2   0.11  0.18  0.05]]
[[ 0.15  0.2   0.1   0.18  0.05]]
[[ 0.15  0.2   0.1   0.17  0.05]]
[[ 0.15  0.19  0.1   0.16  0.05]]
[[ 0.14  0.19  0.09  0.16  0.05]]
[[ 0.14  0.19  0.09  0.16  0.05]]
[[ 0.14  0.18  0.09  0.15  0.05]]
[[ 0.14  0.18  0.09  0.15  0.05]]
[[ 0.13  0.18  0.09  0.15  0.05]]
[[ 0.13  0.17  0.08  0.14  0.05]]
[[ 0.13  0.17  0.08  0.14  0.05]]
[[ 0.13  0.17  0.08  0.14  0.05]]
[[ 0.13  0.17  0.08  0.14  0.05]]
[[ 0.13  0.17  0.08  0.14  0.05]]
[[ 0.13  0.17  0.08  0.14  0.05]]
[[ 0.12  0.17  0.08  0.14  0.05]]
[[ 0.12  0.17  0.08  0.14  0.05]]
[[ 0.12  0.17  0.08  0.13  0.05]]
[[ 0.12  0.17  0.08  0.13  0.05]]
[[ 0.12  0.16  0.08  0.13  0.05]]
[[ 0.12  0.16  0.08  0.13  0.05]]
[[ 0.12  0.16  0.08  0.13  0.05]]
[[ 0.12  0.16  0.08  0.13  0.05]]
[[ 0.12  0.16  0.08  0.13  0.05]]
[[ 0.12  0.16  0.08  0.13  0.05]]
[[ 0.12  0.16  0.08